# Fun with NFL Stats, Bokeh, and Pandas
For a thorough write up, please see my blog post of the same name.

In [2]:
# created with Python 3.6.7
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange, FixedTicker, Legend
from bokeh.io import output_notebook
from collections import Counter
from bokeh.transform import factor_cmap
from bokeh.palettes import Paired, Spectral
import itertools
pd.set_option('display.max_columns', 150)
output_notebook()

Loading BokehJS ...

In [3]:
# load the data
filename = 'NFL Play by Play 2009-2017 (v4).csv'
df = pd.read_csv(filename)

/Users/howard/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df['down'].isnull().sum()
pd.to_numeric(df['down'], errors='coerce').isnull().sum()

61154

In [10]:
# if you want to filter down to just a single team enter it here
team = 'all'

# to see valid team name strings uncomment the next line
# list(set(df['posteam']))

In [11]:
if team.lower() == 'all':
    team_df = df
else:
    team_df = df.loc[df['posteam'] == team]
team_df = team_df.loc[df['down'].notnull()]

In [12]:
# counting play types
all_play_types = Counter(team_df['PlayType'])
all_play_types

Counter({'Pass': 158928,
         'Run': 120624,
         'Punt': 22003,
         'Sack': 10643,
         'Field Goal': 8927,
         'No Play': 21225,
         'QB Kneel': 3529,
         'Spike': 640,
         'Timeout': 12,
         'Kickoff': 2,
         'Half End': 1})

In [13]:
plays_on_down = [Counter(team_df.loc[team_df['down'] == down]['PlayType']) for down in range(1,5)]

In [14]:
# play_on_down is a list of coutners.
# to see the types and count of plays on first down, access the 0th element of the list
plays_on_down[0]

Counter({'Pass': 59329,
         'Run': 65549,
         'No Play': 7905,
         'Field Goal': 258,
         'Sack': 3245,
         'QB Kneel': 2071,
         'Spike': 519,
         'Timeout': 1,
         'Punt': 1})

In [17]:
# getting data ready for plotting in Bokeh
downs = ['1','2','3','4']
plays = ['Pass', 'Run', 'Punt', 'Field Goal']
x = list(itertools.product(downs, plays))
plays_on_down = [Counter(team_df.loc[team_df['down'] == int(down)]['PlayType']) for down in downs]
counts = [plays_on_down[int(down)-1][play] for down, play in x]
source = ColumnDataSource(data=dict(x=x, counts=counts))
TOOLTIPS = [('Down, Play', '@x'),('Count', '@counts')]
p = figure(x_range=FactorRange(*x), plot_height=350, plot_width=750, title='Play by Down',
           toolbar_location=None, tools='hover', tooltips=TOOLTIPS)
p.vbar(x='x', top='counts', width=0.9, source=source, line_color='white',
           fill_color=factor_cmap('x', palette=Spectral[4], factors=plays, start=1, end=2))
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xaxis.axis_label = 'Down'
p.yaxis.axis_label = 'Number of Plays'
p.xgrid.grid_line_color = None
show(p)

In [24]:
# looking at just run and pass plays on third down as a function of yards to go for a first down
y2g = range(1,25)
team_df_d3 = team_df.loc[team_df['down'] == 3]
plays_on_d3 = [Counter(team_df_d3.loc[team_df_d3['ydstogo'] == yrd]['PlayType']) for yrd in y2g]
x = y2g
y_runs = [play['Run'] for play in plays_on_d3]
y_pass = [play['Pass'] for play in plays_on_d3]
p = figure(title='Third Down Play Type by Yards to Go', toolbar_location=None, tools='',
           plot_height=350, plot_width=750)
p.line(x, y_pass, color='#2b83ba', legend='Pass', line_width=4)
p.line(x, y_runs, color='#abdda4', legend='Run', line_width=4)
p.legend.location = 'top_right'
p.xaxis.axis_label = 'Yards to Go'
p.yaxis.axis_label = 'Number of Plays'
show(p)

In [25]:
# looking at pass/run ratio on third down as a function of yards to go for a first down
x = y2g
y = [play['Pass']/play['Run'] for play in plays_on_d3]
p = figure(title='Third Down Pass/Run Ratio by Yards to Go', toolbar_location=None, tools='',
           plot_height=350, plot_width=750)
p.line(x, y, color='#2b83ba', line_width=4)
p.xaxis.axis_label = 'Yards to Go'
p.yaxis.axis_label = 'Pass/Run Ratio'
show(p)

# Looking at plays as a fucntion of yard line

In [27]:
yrdline = range(0,101)
plays_on_yrd = [Counter(team_df.loc[team_df['yrdline100'] == yrd]['PlayType']) for yrd in yrdline]

In [28]:
x = range(0,101)
y_pass = [play['Pass'] for play in plays_on_yrd]
y_runs = [play['Run'] for play in plays_on_yrd]
y_punt = [play['Punt'] for play in plays_on_yrd]
y_fg = [play['Field Goal'] for play in plays_on_yrd]
p = figure(title='Plays by Yard Line', toolbar_location=None, tools='',
           plot_height=350, plot_width=750, x_range=(1,99))
p.line(x, y_pass, color='#2b83ba', legend='Pass', line_width=4)
p.line(x, y_runs, color='#abdda4', legend='Run', line_width=4)
p.line(x, y_punt, color='#fdae61', legend='Punt', line_width=4)
p.line(x, y_fg, color='#d7191c', legend='Field Goal', line_width=4)
p.legend.location = 'top_right'
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Number of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker
show(p)

## Normalizing the plays by the total number of plays at each yard line

In [30]:
def total_plays(i, play, cou):
    total_plays = sum([cou[i][play] for play in plays])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
y_pass = [play['Pass']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line (normalized by total number of plays by yard line)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### First down only

In [31]:
from bokeh.models import Legend
def total_plays(i, play, cou):
    total_plays = sum([cou[i][play] for play in plays])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d1 = team_df.loc[team_df['down'] == 1]
plays_on_yrd = [Counter(team_df_d1.loc[team_df_d1['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on First Down (normalized by total number of plays by yard line)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Second down only

In [33]:
def total_plays(i, play, cou):
    total_plays = sum([cou[i][play] for play in plays])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d2 = team_df.loc[team_df['down'] == 2]
plays_on_yrd = [Counter(team_df_d2.loc[team_df_d2['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Second Down (normalized by total number of plays by yard line)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Third down only

In [34]:
def total_plays(i, play, cou):
    total_plays = sum([cou[i][play] for play in plays])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d3 = team_df.loc[team_df['down'] == 3]
plays_on_yrd = [Counter(team_df_d3.loc[team_df_d3['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Third Down (normalized by total number of plays by yard line)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Fourth down only

In [35]:
def total_plays(i, play, cou):
    total_plays = sum([cou[i][play] for play in plays])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d4 = team_df.loc[team_df['down'] == 4]
plays_on_yrd = [Counter(team_df_d4.loc[team_df_d4['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, play, plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Fourth Down (normalized by total number of plays by yard line)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

## Normalizing by the total plays of a given type

In [36]:
def total_plays(i, play, cou):
    total_plays = sum([p[play] for p in cou])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
plays_on_yrd = [Counter(team_df.loc[team_df['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, 'Pass', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, 'Run', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, 'Punt', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, 'Field Goal', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line (normalized by total number of plays of given type)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### First down only

In [37]:
def total_plays(i, play, cou):
    total_plays = sum([p[play] for p in cou])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d1 = team_df.loc[team_df['down'] == 1]
plays_on_yrd = [Counter(team_df_d1.loc[team_df_d1['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, 'Pass', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, 'Run', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, 'Punt', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, 'Field Goal', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on First Down (normalized by total number of plays of given type)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99), y_range=(-0.005,0.1))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Second down only

In [38]:
def total_plays(i, play, cou):
    total_plays = sum([p[play] for p in cou])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d1 = team_df.loc[team_df['down'] == 2]
plays_on_yrd = [Counter(team_df_d2.loc[team_df_d2['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, 'Pass', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, 'Run', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, 'Punt', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, 'Field Goal', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Second Down (normalized by total number of plays of given type)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Third down only

In [39]:
def total_plays(i, play, cou):
    total_plays = sum([p[play] for p in cou])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d1 = team_df.loc[team_df['down'] == 3]
plays_on_yrd = [Counter(team_df_d3.loc[team_df_d3['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, 'Pass', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, 'Run', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, 'Punt', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, 'Field Goal', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Third Down (normalized by total number of plays of given type)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99), y_range=(-0.005,0.1))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)

### Fourth down only

In [40]:
def total_plays(i, play, cou):
    total_plays = sum([p[play] for p in cou])
    if total_plays == 0:
        return 1
    else:
        return total_plays

x = range(0,101)
team_df_d1 = team_df.loc[team_df['down'] == 4]
plays_on_yrd = [Counter(team_df_d4.loc[team_df_d4['yrdline100'] == yrd]['PlayType']) for yrd in x]
y_pass = [play['Pass']/total_plays(i, 'Pass', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_runs = [play['Run']/total_plays(i, 'Run', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_punt = [play['Punt']/total_plays(i, 'Punt', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
y_fg = [play['Field Goal']/total_plays(i, 'Field Goal', plays_on_yrd) for i, play in enumerate(plays_on_yrd)]
p = figure(title='Plays by Yard Line on Fourth Down (normalized by total number of plays of given type)', toolbar_location=None, tools='hover',
           plot_height=350, plot_width=750, x_range=(1,99))
l1 = p.line(x, y_pass, color='#2b83ba', line_width=4)
l2 = p.line(x, y_runs, color='#abdda4', line_width=4)
l3 = p.line(x, y_punt, color='#fdae61', line_width=4)
l4 = p.line(x, y_fg, color='#d7191c', line_width=4)
p.xaxis.axis_label = 'Yard Line (100 is team\'s own goal line)'
p.yaxis.axis_label = 'Fraction of Plays'
p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 5)))
p.xgrid.ticker = p.xaxis[0].ticker

legend = Legend(items=[
    ('Pass'   , [l1]),
    ('Run' , [l2]),
    ('Punt' , [l3]),
    ('Field Goal', [l4]),
], location=(0, +100))

p.add_layout(legend, 'right')
show(p)